In [87]:
import pandas as pd
import numpy as np

In [88]:
df = pd.read_csv('../database/clean_train_data_v2.csv', index_col = 0)

In [89]:
df.head()

,LATITUDE,LONGITUDE,ELEVATION,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPresentWeatherType,HourlyPressureTendency,HourlyRelativeHumidity,HourlySkyConditions,...,HourlyWetBulbTemperature,HourlyWindDirection,HourlyWindSpeed,YEAR,MONTH,DAY,HOUR,MINUTES,SECONDS,TARGETS
0,26.760594,80.889339,124.96,NaN,61.0,100.0,||HZ,9.0,27.0,FEW:01 98,...,NaN,90.0,10.0,2013,5,5,11,30,0,2.49
1,26.760594,80.889339,124.96,30.06,55.0,61.0,NaN,NaN,83.0,SCT:04 20 SCT:04 100,...,NaN,NaN,2.0,2019,1,23,22,0,0,1.24
2,25.160219,75.845631,273.10,NaN,51.0,67.0,NaN,9.0,57.0,NaN,...,NaN,230.0,5.0,2005,2,17,8,30,0,2.49
3,26.760594,80.889339,124.96,NaN,41.0,94.0,||DU,9.0,16.0,NaN,...,NaN,0.0,0.0,1984,5,20,8,30,0,2.49
4,25.033333,88.133333,31.00,NaN,55.0,82.0,NaN,9.0,39.0,NaN,...,NaN,320.0,2.0,2016,11,16,11,30,0,2.49


In [90]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 934807 entries, 0 to 934806
Data columns (total 22 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   LATITUDE                   934807 non-null  float64
 1   LONGITUDE                  934807 non-null  float64
 2   ELEVATION                  934807 non-null  float64
 3   HourlyAltimeterSetting     217039 non-null  float64
 4   HourlyDewPointTemperature  916829 non-null  float64
 5   HourlyDryBulbTemperature   923010 non-null  float64
 6   HourlyPresentWeatherType   398691 non-null  object 
 7   HourlyPressureTendency     586427 non-null  float64
 8   HourlyRelativeHumidity     916208 non-null  float64
 9   HourlySkyConditions        507788 non-null  object 
 10  HourlySeaLevelPressure     670809 non-null  float64
 11  HourlyStationPressure      24403 non-null   float64
 12  HourlyWetBulbTemperature   24222 non-null   float64
 13  HourlyWindDirection        86

In [91]:
df.SECONDS.value_counts()

0    934807
Name: SECONDS, dtype: int64

In [92]:
df.drop(['SECONDS'], axis = 1, inplace = True)

In [93]:
df.sort_values(['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTES'], inplace = True)

In [94]:
df.head()

,LATITUDE,LONGITUDE,ELEVATION,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPresentWeatherType,HourlyPressureTendency,HourlyRelativeHumidity,HourlySkyConditions,...,HourlyStationPressure,HourlyWetBulbTemperature,HourlyWindDirection,HourlyWindSpeed,YEAR,MONTH,DAY,HOUR,MINUTES,TARGETS
118582,27.155831,77.960892,167.94,NaN,NaN,NaN,NaN,NaN,NaN,FEW:02 15 FEW:01,...,NaN,NaN,113.0,3.0,1942,8,24,7,30,99.42
755013,27.155831,77.960892,167.94,NaN,NaN,NaN,NaN,NaN,NaN,FEW:02 15 FEW:01,...,NaN,NaN,113.0,8.0,1942,8,24,8,30,99.42
824694,27.155831,77.960892,167.94,NaN,NaN,NaN,NaN,NaN,NaN,FEW:02 15 FEW:01,...,NaN,NaN,135.0,8.0,1942,8,24,9,30,99.42
830375,27.155831,77.960892,167.94,NaN,NaN,NaN,NaN,NaN,NaN,SCT:04 15,...,NaN,NaN,135.0,9.0,1942,8,24,10,30,99.42
680093,27.155831,77.960892,167.94,NaN,NaN,NaN,NaN,NaN,NaN,BKN:06 FEW:01,...,NaN,NaN,135.0,8.0,1942,8,24,11,30,99.42


In [102]:
def fill_interpolate_temporal(date_sorted_data):
    i = 0
    t = 0
    while (i < len(date_sorted_data)):
        prev_val = date_sorted_data[i]
        j = i+1
        while np.isnan(date_sorted_data[j]):
            j = j+1
        next_val = date_sorted_data[j]
        if np.isnan(prev_val):
            for k in range(i, j):
                date_sorted_data[k] = next_val
        else:
            delta = (next_val - prev_val) / (j - i)
            for k in range(i, j):
                date_sorted_data[k] = prev_val + (k - i)*delta
        i = j
        t = t+1
        if (t%10000==0):
            print(t)
    return date_sorted_data

In [110]:
df.HourlyDryBulbTemperature[0:60000].isnull().sum()

0

In [104]:
fill_interpolate_temporal(df.HourlyDryBulbTemperature).isnull().sum()

c:\users\hp\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000


KeyError: 934807

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 934807 entries, 0 to 934806
Data columns (total 22 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   LATITUDE                   934807 non-null  float64
 1   LONGITUDE                  934807 non-null  float64
 2   ELEVATION                  934807 non-null  float64
 3   HourlyAltimeterSetting     217039 non-null  float64
 4   HourlyDewPointTemperature  916829 non-null  float64
 5   HourlyDryBulbTemperature   923010 non-null  float64
 6   HourlyPresentWeatherType   398691 non-null  object 
 7   HourlyPressureTendency     586427 non-null  float64
 8   HourlyRelativeHumidity     916208 non-null  float64
 9   HourlySkyConditions        507788 non-null  object 
 10  HourlySeaLevelPressure     670809 non-null  float64
 11  HourlyStationPressure      24403 non-null   float64
 12  HourlyWetBulbTemperature   24222 non-null   float64
 13  HourlyWindDirection        86

In [126]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [122]:
X = df[['LATITUDE', 'LONGITUDE', 'ELEVATION', 'YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTES']]
y = df['TARGETS']
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True)
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [123]:
y_pred = lr.predict(X_test)

In [124]:
r2_score(y_pred, y_test)

-4.963015335921075

In [127]:
mean_squared_error(y_pred, y_test)

75.80544463149002

In [129]:
df.LATITUDE.value_counts()

26.760594    315313
23.287828    119271
26.739708    101532
25.160219    100011
27.155831     86740
28.000000     50678
25.233333     36070
27.566667     21667
28.250000     16416
27.050000     14668
32.666667     13673
25.033333     13428
27.333333      6697
27.616667      6063
24.466667      5456
27.883333      4201
25.983333      3100
26.516667      3079
27.066667      3020
25.166667      2979
23.316667      2936
24.266667      2781
34.433333      1949
25.666667      1547
27.900000      1484
30.400000        20
25.750000        11
29.850000        10
28.416667         3
31.333333         3
28.633333         1
Name: LATITUDE, dtype: int64